In [ ]:
import pandas as pd
import numpy as np

import datetime as dt
import os

import random
import string

In [ ]:
import spikeinterface.full as si  # import core only
import spikeinterface.preprocessing as spre

from spikeinterface.sortingcomponents.peak_detection import detect_peaks

from typing import Tuple, List
from probeinterface import Probe

def load_recording_from_raw(root: str, sample_base: str, well: Tuple[int, int], time_samplings_to_mask: List[Tuple[float, float]]):

    traces_list = []
    channel_ids = []

    df = pd.read_csv(f'{root}/{sample_base}/{sample_base}.info', index_col=0, names=['index', 'value'], sep='\t')
    sampling_frequency = df.loc['SamplingFrequency', 'value']
    voltage_scale = np.abs(df.loc['VoltageScale', 'value'])

    # We choose 10 here because in 64-electrode MEAs the range would be up to 9. 
    # Since the time required for the non-existing electrodes is small, we don't mine using a larger number.
    for Erow in range(1,10):  
        for Ecol in range(1,10):
            filename = f'{root}/{sample_base}/{well[0]}-{well[1]}-{Erow}-{Ecol}_voltageRaw'
            is_txt, is_gzip = os.path.exists(f'{filename}.txt'), os.path.exists(f'{filename}.txt.gz') 

            if is_txt or is_gzip:
                channel_ids.append(f'{Erow}-{Ecol}')
                
                if is_txt:
                    list_voltages = np.loadtxt(f'{filename}.txt')
                elif is_gzip:
                    list_voltages = np.loadtxt(f'{filename}.txt.gz')

                traces_list.append(list_voltages)
            

    trace_array = np.asarray(traces_list).transpose() / voltage_scale

    for time_sampling in time_samplings_to_mask:
        t0 = int(time_sampling[0] * sampling_frequency)
        tf = int(time_sampling[1] * sampling_frequency)
        trace_array[t0:tf, :] = 0

    sample_recording = si.NumpyRecording(
        traces_list=[trace_array],
        sampling_frequency=sampling_frequency,
        channel_ids=np.asarray(channel_ids)
    )

    sample_recording.set_property('group', [0] * len(channel_ids))
    sample_recording.is_dumpable = True  # This is necessary for some options later, like spike sorting

    return sample_recording


def load_probe_recording(recording: si.NumpyRecording, type_MEAS: int, ):
    dist_multiplier = 350 if type_MEAS == 16 else 300
    circle_radius = 50

    channel_ids = recording.get_channel_ids()

    positions = np.zeros((len(channel_ids), 2), dtype=float)
    contact_vector = []
    for channel_idx, channel in enumerate(channel_ids):
        x_coord, y_coord = (int(channel.split('-')[0]) - 1) * dist_multiplier, (int(channel.split('-')[1]) - 1) * dist_multiplier
        positions[channel_idx, 1], positions[channel_idx, 0] = x_coord, y_coord
        
        contact_vector.append((0, x_coord,   y_coord, 'circle', circle_radius, '', '', channel_idx, 'um', 1., 0., 0., 1.))

    # later if we are using peak detection, we may need it
    recording.set_channel_locations(locations=positions)

    probe = Probe(ndim=2, si_units='um')
    probe.set_contacts(positions=positions, shapes='circle', shape_params={'radius': circle_radius})
    probe.device_channel_indices = np.arange(len(channel_ids))
    probe.create_auto_shape('rect')

    recording.set_probe(probe)


    # Create contact_vector
    dtypes=[('probe_index', '<i8'), ('x', '<f8'), ('y', '<f8'), ('contact_shapes', '<U64'), 
            ('radius', '<f8'), ('shank_ids', '<U64'), ('contact_ids', '<U64'), ('device_channel_indices', '<i8'), 
            ('si_units', '<U64'), ('plane_axis_x_0', '<f8'), ('plane_axis_x_1', '<f8'), ('plane_axis_y_0', '<f8'), 
            ('plane_axis_y_1', '<f8')]

    recording.set_property('contact_vector', np.asarray(contact_vector, dtype=dtypes))

In [ ]:
def retrieve_peaks(root, sample_base, well):
    session_token = dt.datetime.now().strftime("%y-%m-%d") + '_' + \
                ''.join(random.choice(string.ascii_letters) for i in range(8)) + str(well[0]) + '-' + str(well[1])
    
    recording = load_recording_from_raw(root=root, sample_base=sample_base, well=well, time_samplings_to_mask=[])
    load_probe_recording(recording=recording, type_MEAS=16)
    
    recording_bin = recording.save(n_jobs=16, chunk_duration="1s", folder=f'tmp/bin_{session_token}')

    recording_f = spre.bandpass_filter(recording_bin, freq_min=300, freq_max=5000)

    recording_cmr = spre.common_reference(recording_f, reference='global', operator='median')

    noise_levels = si.get_noise_levels(recording_cmr, return_scaled=False)

    peaks = detect_peaks(recording_cmr,
                        method='locally_exclusive',
                        local_radius_um=450, 
                        detect_threshold=5,
                        noise_levels=noise_levels,
                        )
    
    list_peaks = []
    list_electrodes = []

    for i in range(16):
        list_peaks_i = [peak[0] / recording.sampling_frequency for peak in peaks if peak[1] == i]
        list_peaks += list_peaks_i

        el_x, el_y = i // 4, i % 4
        list_electrodes += [f'{el_x + 1}{el_y + 1}'] * len(list_peaks_i)

    return list_peaks, list_electrodes

## Running cells

In [ ]:
from nb_vars import FOLDER_INPUT_BASE, FOLDER_OUTPUT_BASE

In [ ]:
# TODO: CAMBIAR LAS PROXIMAS CELDAS PARA QUE EL LIST_CONDITIONS SEA UNA TABLA Y SE LEA DE ALGUN LADO
# TODO: INCLUIR UN ID DE EXPERIMENTO PARA LOS ARCHIVOS

**RUN ONLY ONE CELL (100, 200 or 500)**

In [ ]:
#20240220
msec = 100    #pulse duration 100msec

folder_input = f'{FOLDER_INPUT_BASE}/{msec}'
folder_output = f'{FOLDER_OUTPUT_BASE}/{msec}'

os.makedirs(folder_output, exist_ok=True)

list_conditions = [#('Condition', 'Treatment', 'Wells', 'Well_num', 'Replicate', 'Folder') 
                    ('BP',          'PRE',     ['C5', 'C6'],               [35, 36],           [1, 2],         'PRE'), 
                    ('BP',          'POST',    ['C5', 'C6'],               [35, 36],           [1, 2],         'POST'), 
                    ('LINK1',       'PRE',     ['D2'],                     [42],               [1],            'PRE'), 
                    ('LINK1',       'POST',    ['D2'],                     [42],               [1],            'POST'), 
                    ('LINK2',       'PRE',     ['D3', 'D4'],               [43, 44],           [1, 2],         'PRE'), 
                    ('LINK2',       'POST',    ['D3', 'D4'],               [43, 44],           [1, 2],         'POST'),
                    ('LINK3',       'PRE',     ['D5', 'D6'],               [45, 46],           [1, 2],         'PRE'),
                    ('LINK3',       'POST',    ['D5', 'D6'],               [45, 46],           [1, 2],         'POST'),
                    ('BP+LINK1',    'PRE',     ['A1', 'A2', 'A3', 'A4'],   [11, 12, 13, 14],   [1, 2, 3, 4],   'PRE'), 
                    ('BP+LINK1',    'POST',    ['A1', 'A2', 'A3', 'A4'],   [11, 12, 13, 14],   [1, 2, 3, 4],   'POST'), 
                    ('BP+LINK2',    'PRE',     ['B2', 'B3', 'B4'],         [22, 23, 24],       [1, 2, 3],      'PRE'), 
                    ('BP+LINK2',    'POST',    ['B2', 'B3', 'B4'],         [22, 23, 24],       [1, 2, 3],      'POST'), 
                    ('BP+LINK3',    'PRE',     ['C2', 'C3', 'C4'],         [32, 33, 34],       [1, 2, 3],      'PRE'), 
                    ('BP+LINK3',    'POST',    ['C2', 'C3', 'C4'],         [32, 33, 34],       [1, 2, 3],      'POST'), 
                   ]

In [ ]:
#20240422-23
msec = 200

folder_input = f'{FOLDER_INPUT_BASE}/{msec}'
folder_output = f'{FOLDER_OUTPUT_BASE}/{msec}'

os.makedirs(folder_output, exist_ok=True)

list_conditions = [#('Condition', 'Treatment', 'Wells', 'Well_num', 'Replicate', 'Folder') 
                    ('CTRL',        'PRE',     ['A1', 'A2', 'A3'],     [11, 12, 13], [1, 2, 3], 'PRE'), 
                    ('CTRL',        'POST',    ['A1', 'A2', 'A3'],     [11, 12, 13], [1, 2, 3], 'POST'), 
                    ('BP',          'PRE',     ['A4', 'A5', 'A6'],     [14, 15, 16], [1, 2, 3], 'PRE'), 
                    ('BP',          'POST',    ['A4', 'A5', 'A6'],     [14, 15, 16], [1, 2, 3], 'POST'),
                    ('LINK1',       'PRE',     ['B4', 'B5', 'B6'],     [25, 25, 26], [1, 2, 3], 'PRE'), 
                    ('LINK1',       'POST',    ['B4', 'B5', 'B6'],     [25, 25, 26], [1, 2, 3], 'POST'), 
                    ('LINK2',       'PRE',     ['C3', 'C4', 'C6'],     [33, 34, 36], [1, 2, 3], 'PRE'), 
                    ('LINK2',       'POST',    ['C3', 'C4', 'C6'],     [33, 34, 36], [1, 2, 3], 'POST'), 
                    ('LINK3',       'PRE',     ['D4', 'D5', 'D6'],     [44, 45, 46], [1, 2, 3], 'PRE'), 
                    ('LINK3',       'POST',    ['D4', 'D5', 'D6'],     [44, 45, 46], [1, 2, 3], 'POST'), 
                    ('BP+LINK1',    'PRE',     ['B1', 'B2', 'B3'],     [21, 22, 23], [1, 2, 3], 'PRE'), 
                    ('BP+LINK1',    'POST',    ['B1', 'B2', 'B3'],     [21, 22, 23], [1, 2, 3], 'POST'), 
                    ('BP+LINK2',    'PRE',     ['C1', 'C2', 'C5'],     [21, 32, 35], [1, 2, 3], 'PRE'), 
                    ('BP+LINK2',    'POST',    ['C1', 'C2', 'C5'],     [21, 32, 35], [1, 2, 3], 'POST'), 
                    ('BP+LINK3',    'PRE',     ['D1', 'D2', 'D3'],     [41, 42, 43], [1, 2, 3], 'PRE'), 
                    ('BP+LINK3',    'POST',    ['D1', 'D2', 'D3'],     [41, 42, 43], [1, 2, 3], 'POST'), 
                   ]

In [ ]:
#20240429-30
msec = 500

folder_input = f'{FOLDER_INPUT_BASE}/{msec}'
folder_output = f'{FOLDER_OUTPUT_BASE}/{msec}'

os.makedirs(folder_output, exist_ok=True)

list_conditions = [#('Condition', 'Treatment', 'Wells', 'Well_num', 'Replicate', 'Folder') 
                    ('CTRL',        'PRE',      ['A1', 'A2', 'A3'], ['11', '12', '13'], [1, 2, 3], 'PRE'), 
                    ('CTRL',        'POST',     ['A1', 'A2', 'A3'], ['11', '12', '13'], [1, 2, 3], 'POST'),
                    ('BP',          'PRE',      ['A4', 'A5', 'A6'], ['14', '15', '16'], [1, 2, 3], 'PRE'), 
                    ('BP',          'POST',     ['A4', 'A5', 'A6'], ['14', '15', '16'], [1, 2, 3], 'POST'), 
                    ('LINK1',       'PRE',      ['B1', 'B2', 'B3'], ['21', '22', '23'], [1, 2, 3], 'PRE'), 
                    ('LINK1',       'POST',     ['B1', 'B2', 'B3'], ['21', '22', '23'], [1, 2, 3], 'POST'),
                    ('LINK2',       'PRE',      ['C1', 'C2', 'C3'], ['31', '32', '33'], [1, 2, 3], 'PRE'), 
                    ('LINK2',       'POST',     ['C1', 'C2', 'C3'], ['31', '32', '33'], [1, 2, 3], 'POST'),
                    ('LINK3',       'PRE',      ['D1', 'D2', 'D3'], ['41', '42', '43'], [1, 2, 3], 'PRE'), 
                    ('LINK3',       'POST',     ['D1', 'D2', 'D3'], ['41', '42', '43'], [1, 2, 3], 'POST'),
                    ('BP+LINK1',    'PRE',      ['B4', 'B5', 'B6'], ['24', '25', '26'], [1, 2, 3], 'PRE'), 
                    ('BP+LINK1',    'POST',     ['B4', 'B5', 'B6'], ['24', '25', '26'], [1, 2, 3], 'POST'),
                    ('BP+LINK2',    'PRE',      ['C4', 'C5', 'C6'], ['34', '35', '36'], [1, 2, 3], 'PRE'), 
                    ('BP+LINK2',    'POST',     ['C4', 'C5', 'C6'], ['34', '35', '36'], [1, 2, 3], 'POST'),
                    ('BP+LINK3',    'PRE',      ['D4', 'D5', 'D6'], ['44', '45', '46'], [1, 2, 3], 'PRE'), 
                    ('BP+LINK3',    'POST',     ['D4', 'D5', 'D6'], ['44', '45', '46'], [1, 2, 3], 'POST'),
                   ]

In [ ]:
#20240607
#linker = 5uM
#BP1 = 0.02nM
#BP2 = 0.05nM

WELL_ = 1

folder_input = f'{FOLDER_INPUT_BASE}/{WELL_}'
folder_output = f'{FOLDER_OUTPUT_BASE}/{WEll_}'

os.makedirs(folder_output, exist_ok=True)

list_conditions = [#('Condition', 'Treatment', 'Wells', 'Well_num', 'Replicate', 'Folder') 
                    ('CTRL_100msec',        'PRE',      ['A1', 'A2'], ['11', '12'], [1, 2], 'PRE'), 
                    ('CTRL_100msec',        'POST',     ['A1', 'A2'], ['11', '12'], [1, 2], 'POST'),
                    ('LINK_100msec',        'PRE',      ['A3', 'A4'], ['13', '14'], [1, 2], 'PRE'), 
                    ('LINK_100msec',        'POST',     ['A3', 'A4'], ['13', '14'], [1, 2], 'POST'), 
                    ('BP1_100msec',         'PRE',      ['B1', 'B2'], ['21', '22'], [1, 2], 'PRE'), 
                    ('BP1_100msec',         'POST',     ['B1', 'B2'], ['21', '22'], [1, 2], 'POST'),
                    ('BP1_200msec',         'PRE',      ['B3', 'B4'], ['23', '24'], [1, 2], 'PRE'), 
                    ('BP1_200msec',         'POST',     ['B3', 'B4'], ['23', '24'], [1, 2], 'POST'),
                    ('BP1_500msec',         'PRE',      ['C1', 'C2'], ['31', '32'], [1, 2], 'PRE'), 
                    ('BP1_500msec',         'POST',     ['C1', 'C2'], ['31', '32'], [1, 2], 'POST'),
                    ('BP2_100msec',         'PRE',      ['C3', 'C4'], ['33', '34'], [1, 2], 'PRE'), 
                    ('BP2_100msec',         'POST',     ['C3', 'C4'], ['33', '34'], [1, 2], 'POST'),
                    ('BP2_200msec',         'PRE',      ['D1', 'D2'], ['41', '42'], [1, 2], 'PRE'), 
                    ('BP2_200msec',         'POST',     ['D1', 'D2'], ['41', '42'], [1, 2], 'POST'),
                    ('BP2_500msec',         'PRE',      ['D3', 'D4'], ['43', '44'], [1, 2], 'PRE'), 
                    ('BP2_500msec',         'POST',     ['D3', 'D4'], ['43', '44'], [1, 2], 'POST')
                   ]

In [ ]:
#20240607
#linker = 5uM
#BP1 = 0.02nM
#BP2 = 0.05nM

WELL_ = 2

folder_input = f'{FOLDER_INPUT_BASE}/{WELL_}'
folder_output = f'{FOLDER_OUTPUT_BASE}/{WELL_}'

os.makedirs(folder_output, exist_ok=True)

list_conditions = [#('Condition', 'Treatment', 'Wells', 'Well_num', 'Replicate', 'Folder') ,
                    ('LINK+BP1_100msec',    'PRE',      ['A2', 'A3'], ['12', '13'], [1, 2], 'PRE'), 
                    ('LINK+BP1_100msec',    'POST',     ['A2', 'A3'], ['12', '13'], [1, 2],  'POST'),
                    ('LINK+BP1_200msec',    'PRE',      ['B2', 'B4'], ['22', '24'], [1, 2], 'PRE'), 
                    ('LINK+BP1_200msec',    'POST',     ['B2', 'B4'], ['22', '24'], [1, 2],  'POST'),
                    ('LINK+BP1_500msec',    'PRE',      ['B5', 'C1'], ['25', '31'], [1, 2], 'PRE'), 
                    ('LINK+BP1_500msec',    'POST',     ['B5', 'C1'], ['25', '31'], [1, 2],  'POST'),
                    ('LINK+BP2_100msec',    'PRE',      ['C3', 'C4'], ['33', '34'], [1, 2], 'PRE'), 
                    ('LINK+BP2_100msec',    'POST',     ['C3', 'C4'], ['33', '34'], [1, 2],  'POST'),
                    ('LINK+BP2_200msec',    'PRE',      ['C5', 'C6'], ['35', '36'], [1, 2], 'PRE'), 
                    ('LINK+BP2_200msec',    'POST',     ['C5', 'C6'], ['35', '36'], [1, 2],  'POST'),
                    ('LINK+BP2_500msec',    'PRE',      ['D3', 'D5'], ['43', '45'], [1, 2], 'PRE'), 
                    ('LINK+BP2_500msec',    'POST',     ['D3', 'D5'], ['43', '45'], [1, 2],  'POST'),
                   ]

**||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||**

In [ ]:
df_peaks = pd.DataFrame(columns=['condition', 'treatment', 'well', 'well_num', 'replicate', 'electrode', 'time'])

for condition, treatment, list_wells, list_wells_num, replicates, sample_base in list_conditions:
    for well, well_num, replicate in zip(list_wells, list_wells_num, replicates):
        print(condition, treatment, well, well_num, replicate, sample_base)
        list_peak_times, list_electrodes = retrieve_peaks(root=folder_input, sample_base=sample_base, well=(int(well_num[0]), int(well_num[1])))
        
        df_peaks_i = pd.DataFrame({'condition': [condition] * len(list_peak_times), 
                                    'treatment': [treatment] * len(list_peak_times), 
                                    'well': [well] * len(list_peak_times), 
                                    'well_num': [well_num] * len(list_peak_times), 
                                    'replicate': [replicate] * len(list_peak_times), 
                                    'electrode': list_electrodes, 
                                    'time': list_peak_times
                                    })
        
        df_peaks = pd.concat([df_peaks, df_peaks_i]).reset_index(drop=True)
        print(len(df_peaks))

In [ ]:
df_peaks.to_csv(f'{folder_output}/df_peaks_full_well{WELL_}.csv', index=False)